# Analyzing SD vendor checkbook data

A starting point for loading data from [the state's vendor checkbook CSV files](https://open.sd.gov/vendor.aspx) and analyzing the data in pandas.

In [32]:
from datetime import datetime
from glob import glob

import pandas as pd
from get_latest_data import CSV_READ_SETTINGS

In [33]:
# no scientific notation for big numbers
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [34]:
# uncomment and run to refresh local csv files
# %run get_latest_data

In [35]:
df = pd.concat([pd.read_csv(x, **CSV_READ_SETTINGS) for x in glob(f'data/*.csv')])

In [36]:
df.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt
0,08,2021-07-27,082200967,VNDR 12623255,12623255,NaN,2021-09-30,726053,1540.00
1,08,2021-07-27,082200967,BRANDI MCCUEN,12623255,NaN,2021-09-30,726053,1540.00
2,08,2021-09-15,22SC08B306210915,HUMAN SERVICE AGENCY,12031085,NaN,2021-09-29,761186,1894.29
3,11,2021-07-30,35467,BANNER & ASSOCIATES INC,12120999,01,2021-09-29,770479,1287.24
4,08,2021-06-21,22SC08A305-JUN8,AVERA MCKENNAN,12029758,86,2021-09-29,766140,73.67


In [37]:
df.sort_values(
    'ap_payment_date',
    ascending=False,
    inplace=True
)

In [38]:
f'Record count: {len(df):,}'

'Record count: 936,840'

In [39]:
df.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt
1869,19,2022-08-09,DCM37313783X,PEACE OF MIND COUNSELING,12474554,NaN,2024-01-19,447737,12.03
912,03,2024-01-02,0865531,SUDS & DUDS INC,12049672,NaN,2024-01-19,444796,14.46
882,18,2023-12-21,S103358889.001,DAKOTA SUPPLY GROUP,12027834,NaN,2024-01-19,444644,29.85
881,18,2024-01-01,21082000021,ARAMARK SERVICES INC,12126032,02,2024-01-19,444601,87335.82
880,18,2024-01-03,142670,CONTROL INSTALLATIONS OF IOWA,12580487,01,2024-01-19,444624,361.84


### Explore payments to a specific vendor

As an example, let's check out payments to vendors containing the names of a few cities in the Northern Hills.

In [46]:
# show me records where the vendor name contains any of these city names
df[df['vendor_name'].str.contains('spearfish|lead|deadwood|whitewood', case=False)]

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name
38,12,2024-01-11,M1240203,SPEARFISH SCHOOL DISTRICT 40-2,12031379,01,2024-01-19,M449589,19232.09,12,EDUCATION
145,12,2024-01-11,M1240204,LEAD-DEADWOOD SCHOOL DIST 40-1,12031442,02,2024-01-19,M449590,1677.66,12,EDUCATION
715,014,2024-01-16,02932438,SPEARFISH ECONOMIC DEV CORP,12302552,NaN,2024-01-19,447759,270.60,014,BUREAU OF HUMAN RESOURCES
739,12,2024-01-11,M1240190,SPEARFISH SCHOOL DISTRICT 40-2,12031379,01,2024-01-19,M449576,12012.00,12,EDUCATION
909,19,2024-01-16,02932438,SPEARFISH ECONOMIC DEV CORP,12302552,NaN,2024-01-19,447759,269.50,19,HUMAN SERVICES
...,...,...,...,...,...,...,...,...,...,...,...
936037,02,2020-06-30,M020BTR024,SPEARFISH-CITY OF,12055082,NaN,2020-07-01,NaN,565773.78,02,REVENUE
936230,02,2020-06-30,M020BTR024,WHITEWOOD-CITY OF,12055252,NaN,2020-07-01,NaN,19378.47,02,REVENUE
936260,02,2020-06-30,M020BTR024,LEAD-CITY OF,12054774,NaN,2020-07-01,NaN,75329.23,02,REVENUE
936543,010,2020-06-23,01X1818-10805,LEAD-CITY OF,12054774,NaN,2020-07-01,NaN,75507.77,010,GOVERNOR'S OFFICE


In [47]:
# drop some vendor numbers of interest into a list
vendor_ids = [
    '12054774',
    '12054391',
    '12055082',
    '12055252'
]

In [50]:
# filter into a new df
df_northern_hills = df[df['vendor_number'].isin(vendor_ids)]

In [51]:
df_northern_hills.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name
2489,18,2024-01-11,01001785-2024-01,CITY OF SPEARFISH,12055082,NaN,2024-01-17,446460,144.09,18,CORRECTIONS
3491,11,2024-01-08,1310649010124,CITY OF DEADWOOD,12054391,NaN,2024-01-17,445954,168.92,11,TRANSPORTATION
3613,02,2024-01-08,024GAM153,CITY OF DEADWOOD,12054391,01,2024-01-12,444170,27981.30,02,REVENUE
3723,02,2024-01-12,M024ALC002,CITY OF SPEARFISH,12055082,NaN,2024-01-12,M445604,22100.50,02,REVENUE
3963,02,2024-01-12,M024ALC002,CITY OF DEADWOOD,12054391,NaN,2024-01-12,M445604,2095.32,02,REVENUE


### Break down spending by month

Add a `yearmonth` column to allow grouping by month.

In [52]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html
df['yearmonth'] = df['ap_payment_date'].dt.strftime('%Y%m')

In [54]:
df.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name,yearmonth
0,19,2022-08-09,DCM37313783X,PEACE OF MIND COUNSELING,12474554,NaN,2024-01-19,447737,12.03,19,HUMAN SERVICES,202401
1,03,2024-01-02,0865531,SUDS & DUDS INC,12049672,NaN,2024-01-19,444796,14.46,03,AGRICULTURE & NAT. RESOURCES,202401
2,18,2023-12-21,S103358889.001,DAKOTA SUPPLY GROUP,12027834,NaN,2024-01-19,444644,29.85,18,CORRECTIONS,202401
3,18,2024-01-01,21082000021,ARAMARK SERVICES INC,12126032,02,2024-01-19,444601,87335.82,18,CORRECTIONS,202401
4,18,2024-01-03,142670,CONTROL INSTALLATIONS OF IOWA,12580487,01,2024-01-19,444624,361.84,18,CORRECTIONS,202401


In [57]:
# pivot table to show spending by vendor by month
pivot_by_vendor_by_month = pd.pivot_table(df,
                              index='vendor_number',
                              columns='yearmonth',
                              values='amt',
                              aggfunc='sum').reset_index().fillna(0.0)

In [58]:
pivot_by_vendor_by_month.head()

yearmonth,vendor_number,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401
0,12001820,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,12001823,53353.87,52533.01,52723.84,52346.98,52186.30,52215.97,51974.41,52402.83,51845.38,...,52808.74,52722.21,52634.96,52797.62,50719.99,51263.86,51487.60,51598.36,51683.68,51926.75
2,12001827,0.00,0.00,0.00,0.00,0.00,0.00,675.00,0.00,736.00,...,0.00,0.00,0.00,775.00,0.00,0.00,0.00,675.00,0.00,0.00
3,12001831,3029.28,10061.58,11085.74,4028.23,0.00,8776.72,6357.01,3394.56,5616.32,...,11398.79,4973.39,62.89,7451.23,6793.89,11144.29,4977.55,9219.84,4977.55,11461.39
4,12001907,102.04,0.00,153.06,0.00,1533.67,0.00,1020.41,0.00,90462.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [59]:
# pivot table to show spending by agency by month
pivot_by_agency_by_month = pd.pivot_table(df,
                              index='agency_name',
                              columns='yearmonth',
                              values='amt',
                              aggfunc='sum').reset_index().fillna(0.0)

In [60]:
pivot_by_agency_by_month.head()

yearmonth,agency_name,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401
0,AGRICULTURE & NAT. RESOURCES,4553984.51,5459280.11,3184810.95,4564255.96,4019103.13,4723435.97,2674370.70,3198881.83,2096449.41,...,8111441.90,19787387.03,22155684.54,16482685.73,22738320.56,15702085.32,16251494.16,13377178.39,15318495.63,8626240.98
1,ATTORNEY GENERAL,957625.66,735497.69,623860.35,785921.39,506460.41,1007728.01,459006.43,853824.42,793472.82,...,386909.35,703842.08,748594.04,769320.00,2706801.00,1192855.58,568347.10,461635.59,1054679.49,586896.87
2,BOARD OF REGENTS,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1028.97,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,BUREAU OF ADMINISTRATION,3290216.74,2439932.15,3072094.16,3313497.95,4101399.72,4510967.20,2326289.87,3640654.57,2937420.32,...,3520392.77,4460971.78,6301605.31,3118755.48,3452070.02,4095114.22,3853794.81,5601987.19,5851664.85,3227717.21
4,BUREAU OF FINANCE & MANAGEMENT,293867.53,29844357.42,16697248.80,45557236.23,26539516.40,284083649.00,251454551.14,66176055.60,10850474.69,...,156091.72,687136.71,951994.48,292883.90,567357.63,195620.09,208931.80,202469.54,247913.15,68122.71


In [61]:
# sort descending by totals for the latest month, which is the last column in the df
pivot_by_agency_by_month.sort_values(pivot_by_agency_by_month.columns[-1], ascending=False).head()

yearmonth,agency_name,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401
22,REVENUE,80743795.95,61222012.16,38239412.77,61372287.64,36028745.61,39450414.97,98740129.01,32738523.57,36940879.35,...,50329589.88,69049528.44,54930087.46,86715415.66,61949976.59,61801795.42,68978106.13,50735116.39,58965811.58,56832405.00
8,EDUCATION,65974788.11,65500606.66,116304643.26,68332112.21,70058588.75,78934670.42,73796056.09,68137473.81,69711016.59,...,80418878.36,100912256.14,98416390.12,98890224.39,85091277.62,77240260.87,108070968.03,105739746.19,102249974.39,39141025.56
29,TRANSPORTATION,112242554.86,80742028.73,85169178.59,88238239.08,44262726.16,49199800.15,25372804.40,17977873.11,57502298.39,...,27538179.72,103695621.42,93820500.63,102958419.46,128560665.05,124376899.13,94462658.87,72117245.71,60597755.95,26353042.53
19,PUBLIC SAFETY,12078053.21,10347085.42,8365622.41,14574360.12,6813429.07,11309391.00,7175636.85,7270727.91,6679994.69,...,3445346.11,16356759.31,14887337.21,10030138.84,8943409.37,7219191.75,5570065.54,10865325.98,10307963.97,17890593.16
25,SOCIAL SERVICES,8910804.44,14696494.19,9760285.98,15355313.05,11015205.61,11838650.16,14656744.66,6937901.90,12034195.00,...,13570558.78,24712912.72,34131399.88,10446620.55,25353088.61,21181047.34,13999912.75,22548799.77,20035986.88,15447683.92


### Filter to examine spending by one agency

Let's filter to look at spending by the state Department of Education.

In [62]:
# find the code to filter on
df_codes[df_codes['agency_name'].str.contains('education', case=False)]

,agency_code,agency_name
15,12,EDUCATION


In [63]:
education = df[df['agency'] == '12']

In [64]:
education.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name,yearmonth
11,12,2024-01-11,M1240206,SANFORD MEDICAL CENTER,12029851,18,2024-01-19,M449592,18872.00,12,EDUCATION,202401
12,12,2024-01-11,M1240205,THE JOURNEY CHURCH,12015084,NaN,2024-01-19,M449591,725.96,12,EDUCATION,202401
21,12,2024-01-11,M1240207,GREGORY COMMUNITY DAY CARE CTR,12034325,01,2024-01-19,M449593,65.49,12,EDUCATION,202401
23,12,2024-01-11,M1240205,SANFORD MEDICAL CENTER,12029851,18,2024-01-19,M449591,106928.77,12,EDUCATION,202401
24,12,2024-01-11,M1240205,YOUTH & FAMILY SERVICES INC,12056438,05,2024-01-19,M449591,92613.15,12,EDUCATION,202401


In [65]:
len(education)

63551

### Add up spending before and after a given date

E.g., compare spending before and after July 1, 2021.

In [66]:
target_date = datetime(2021, 7, 1)

In [67]:
spending_before_date = df[df['ap_payment_date'] < target_date]
spending_after_date = df[df['ap_payment_date'] >= target_date]

In [68]:
print(f'Before: ${spending_before_date["amt"].sum():,.2f}')
print(f'After: ${spending_after_date["amt"].sum():,.2f}')

Before: $3,801,255,903.43
After: $9,502,844,615.45


### Break down payment frequency to vendors by month

Use case: See if payments went up or down, or stopped or started, to a vendor or a group of vendors at a given month. E.g., a new rule went into effect and you want to see if/how that affected the frequency/amount of spending, but instead of just a before/after sum you want a monthly breakdown of previous spending to get a sense of frequency.

In [69]:
# what's the target date of the change we're looking at?
target_date = datetime(2022, 10, 1)

# get it in a form that matches column names: yyyymm
# c.f. http://strftime.org
target_yearmonth = target_date.strftime('%Y%m')

In [70]:
# goal is to get a list of column names with yearmonth values
# before and after the target_yearmonth

# gonna do this one step at a time

# show column names
pivot_by_vendor_by_month.columns

Index(['vendor_number', '202007', '202008', '202009', '202010', '202011',
       '202012', '202101', '202102', '202103', '202104', '202105', '202106',
       '202107', '202108', '202109', '202110', '202111', '202112', '202201',
       '202202', '202203', '202204', '202205', '202206', '202207', '202208',
       '202209', '202210', '202211', '202212', '202301', '202302', '202303',
       '202304', '202305', '202306', '202307', '202308', '202309', '202310',
       '202311', '202312', '202401'],
      dtype='object', name='yearmonth')

In [71]:
# show column names except the initial vendor_number column
pivot_by_vendor_by_month.columns[1:]

Index(['202007', '202008', '202009', '202010', '202011', '202012', '202101',
       '202102', '202103', '202104', '202105', '202106', '202107', '202108',
       '202109', '202110', '202111', '202112', '202201', '202202', '202203',
       '202204', '202205', '202206', '202207', '202208', '202209', '202210',
       '202211', '202212', '202301', '202302', '202303', '202304', '202305',
       '202306', '202307', '202308', '202309', '202310', '202311', '202312',
       '202401'],
      dtype='object', name='yearmonth')

In [72]:
# figure out the index position of the target_yearmonth in the list of columns
# see list.index() docs https://docs.python.org/3/tutorial/datastructures.html
# n.b., you have to coerce the .columns object to a list() before you can use the .index() method,

target_col_idx = list(pivot_by_vendor_by_month.columns).index(target_yearmonth)

In [73]:
# doublecheck that the column name you're selecting for matches `target_yearmonth`
assert(pivot_by_vendor_by_month.columns[target_col_idx] == target_yearmonth)

In [74]:
# finally, get a list of columns with yearmonth values before ... (minus initial vendor_number col)
pre_date_columns = pivot_by_vendor_by_month.columns[1:target_col_idx]

# ... and after
post_date_columns = pivot_by_vendor_by_month.columns[target_col_idx:]

In [75]:
# next, goal is to add up total spending prior to this month and after
# define a function that will sum spending across a row for the
# selected columns

def sum_totals_yearmonth(row, direction='pre'):
    cols = pre_date_columns
    
    if direction == 'post':
        cols = post_date_columns

    return round(sum([row[x] for x in cols]), 2)

In [76]:
pivot_by_vendor_by_month['pre_date_total'] = pivot_by_vendor_by_month.apply(sum_totals_yearmonth, direction='pre', axis=1)
pivot_by_vendor_by_month['post_date_total'] = pivot_by_vendor_by_month.apply(sum_totals_yearmonth, direction='post',axis=1)

In [77]:
pivot_by_vendor_by_month.head()

yearmonth,vendor_number,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202306,202307,202308,202309,202310,202311,202312,202401,pre_date_total,post_date_total
0,12001820,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10608.70
1,12001823,53353.87,52533.01,52723.84,52346.98,52186.30,52215.97,51974.41,52402.83,51845.38,...,52634.96,52797.62,50719.99,51263.86,51487.60,51598.36,51683.68,51926.75,1398656.23,833794.56
2,12001827,0.00,0.00,0.00,0.00,0.00,0.00,675.00,0.00,736.00,...,0.00,775.00,0.00,0.00,0.00,675.00,0.00,0.00,2155.00,1450.00
3,12001831,3029.28,10061.58,11085.74,4028.23,0.00,8776.72,6357.01,3394.56,5616.32,...,62.89,7451.23,6793.89,11144.29,4977.55,9219.84,4977.55,11461.39,184974.46,115261.44
4,12001907,102.04,0.00,153.06,0.00,1533.67,0.00,1020.41,0.00,90462.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,612680.57,0.00


In [78]:
# make a quick df to look up vendor names by number
vendor_lookup = df[['vendor_number', 'vendor_name']].drop_duplicates(subset=['vendor_number'])

In [79]:
vendor_lookup.head()

,vendor_number,vendor_name
0,12474554,PEACE OF MIND COUNSELING
1,12049672,SUDS & DUDS INC
2,12027834,DAKOTA SUPPLY GROUP
3,12126032,ARAMARK SERVICES INC
4,12580487,CONTROL INSTALLATIONS OF IOWA


In [80]:
# merge with pivoted df
pivoted_with_vendor_names = pd.merge(
    pivot_by_vendor_by_month,
    vendor_lookup,
    how='left',
    on='vendor_number'
)

In [81]:
pivoted_with_vendor_names.head()

,vendor_number,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202307,202308,202309,202310,202311,202312,202401,pre_date_total,post_date_total,vendor_name
0,12001820,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10608.70,LL BEAN INC
1,12001823,53353.87,52533.01,52723.84,52346.98,52186.30,52215.97,51974.41,52402.83,51845.38,...,52797.62,50719.99,51263.86,51487.60,51598.36,51683.68,51926.75,1398656.23,833794.56,UNUM LIFE INS CO OF AMERICA
2,12001827,0.00,0.00,0.00,0.00,0.00,0.00,675.00,0.00,736.00,...,775.00,0.00,0.00,0.00,675.00,0.00,0.00,2155.00,1450.00,NATIONAL RURAL HEALTH ASSN
3,12001831,3029.28,10061.58,11085.74,4028.23,0.00,8776.72,6357.01,3394.56,5616.32,...,7451.23,6793.89,11144.29,4977.55,9219.84,4977.55,11461.39,184974.46,115261.44,IDEXX LABORATORIES INC
4,12001907,102.04,0.00,153.06,0.00,1533.67,0.00,1020.41,0.00,90462.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,612680.57,0.00,BAETE FORSETH HVAC


In [82]:
# make sure you ended up with the same number of records
assert(len(pivot_by_vendor_by_month) == len(pivoted_with_vendor_names))

In [83]:
# filter to get vendors who were paid something in the months before
# the date of interest but not after
some_before_none_after = pivoted_with_vendor_names[(pivoted_with_vendor_names['pre_date_total'] > 0) & (pivoted_with_vendor_names['post_date_total'] == 0)]

print(f'{len(some_before_none_after):,} vendors were paid something before the target month but nothing after')
print()

# starting with the month immediately preceding the target yearmonth,
# count up how many vendors were paid that month -- the
# goal is to get a sense of consecutive payments each month that stopped,
# and it's accomplished by gradually filtering the same dataframe
# with a filter that iterates backward by month

# loop over the list of pre-date columns in reverse
for i, yearmonth in enumerate(reversed(pre_date_columns), 1):

    month_phrase = f'{i} consecutive months'
    
    if i == 1:
        month_phrase = f'the month'
        

    # filter to get vendors who were paid more than 0 in this month
    some_before_none_after = some_before_none_after[some_before_none_after[yearmonth] > 0]
    
    # get the total number in this cohort
    total = len(some_before_none_after)
        
    plural_phrase = 'vendors were'
    
    if total == 1:
        plural_phrase = 'vendor was'

    msg = f'{total:>4,} {plural_phrase} paid something in the {month_phrase} before the target month, but nothing after'
    print(msg)

13,385 vendors were paid something before the target month but nothing after

 404 vendors were paid something in the the month before the target month, but nothing after
  44 vendors were paid something in the 2 consecutive months before the target month, but nothing after
  16 vendors were paid something in the 3 consecutive months before the target month, but nothing after
  13 vendors were paid something in the 4 consecutive months before the target month, but nothing after
   9 vendors were paid something in the 5 consecutive months before the target month, but nothing after
   7 vendors were paid something in the 6 consecutive months before the target month, but nothing after
   6 vendors were paid something in the 7 consecutive months before the target month, but nothing after
   4 vendors were paid something in the 8 consecutive months before the target month, but nothing after
   4 vendors were paid something in the 9 consecutive months before the target month, but nothing aft